### How to build your own catalog
Author: Tadd Bindas

In [1]:
from pathlib import Path

from pyiceberg.catalog import load_catalog

from icefabric_manage import build
from icefabric_tools import find_origin, to_geopandas

First, you will need to define where your local data warehouse will live. This file will be located on disk and requires you to already have some geoparquet files to load

In [2]:
warehouse_path = Path("../data/warehouse")
warehouse_path.mkdir(exist_ok=True)

catalog_settings = {
    'type': 'sql',
    "uri": f"sqlite:///{warehouse_path}/pyiceberg_catalog.db",
    "warehouse": f"file://{warehouse_path}",
}

catalog = load_catalog("hydrofabric", **catalog_settings)
build(catalog, Path("/Users/taddbindas/projects/NGWPC/icefabric/data/parquet"))

Table network already exists. Skipping build
Table nexus already exists. Skipping build
Table flowpath-attributes already exists. Skipping build
Table divides already exists. Skipping build
Table pois already exists. Skipping build
Table flowpath-attributes-ml already exists. Skipping build
Table divide-attributes already exists. Skipping build
Table flowpaths already exists. Skipping build
Table hydrolocations already exists. Skipping build
Table lakes already exists. Skipping build


Once the catalog is created, you can load the table into memory, scan the file based on a query, then return the object via pandas dataframe

In [3]:
network = catalog.load_table("hydrofabric.network")
network.scan().to_pandas()

,id,toid,divide_id,ds_id,mainstem,hydroseq,hf_source,hf_id,lengthkm,areasqkm,tot_drainage_areasqkm,type,vpuid,hf_hydroseq,hf_lengthkm,hf_mainstem,topo,poi_id,hl_uri
0,wb-20469,tnx-1000000125,cat-20469,NaN,2613576.0,20283.0,NOAA Reference Fabric,166196253.0,7.006821,13.96665,72.707401,terminal,01,2613576.0,7.006821,2613576.0,fl-nex,NaN,None
1,wb-20469,tnx-1000000125,cat-20469,NaN,2613576.0,20283.0,NOAA Reference Fabric,166196257.0,7.006821,13.96665,72.707401,terminal,01,2613602.0,0.992107,2613601.0,fl-nex,NaN,None
2,wb-20469,tnx-1000000125,cat-20469,NaN,2613576.0,20283.0,NOAA Reference Fabric,166196256.0,7.006821,13.96665,72.707401,terminal,01,2613601.0,0.940432,2613601.0,fl-nex,NaN,None
3,wb-20479,tnx-1000000697,cat-20479,NaN,2613603.0,20277.0,NOAA Reference Fabric,4599061.0,1.445138,5.88735,42.698700,terminal,01,2613604.0,0.793282,2613603.0,fl-nex,NaN,None
4,wb-20479,tnx-1000000697,cat-20479,NaN,2613603.0,20277.0,NOAA Reference Fabric,4599715.0,1.445138,5.88735,42.698700,terminal,01,2613605.0,0.205191,2613603.0,fl-nex,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3461362,None,cnx-3299511,cat-3299511,NaN,NaN,NaN,NOAA Reference Fabric,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,fl-nex,NaN,None
3461363,None,cnx-3299510,cat-3299510,NaN,NaN,NaN,NOAA Reference Fabric,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,fl-nex,NaN,None
3461364,None,cnx-3299509,cat-3299509,NaN,NaN,NaN,NOAA Reference Fabric,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,fl-nex,NaN,None
3461365,None,cnx-3299508,cat-3299508,NaN,NaN,NaN,NOAA Reference Fabric,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,fl-nex,NaN,None


Now that we have these tables in memory, we can use predefined functions/services to get more insight from our data/methods

### Finding an Origin point

In [6]:
find_origin(network_table=network, identifier=18471000, id_type="comid")

,id,toid,vpuid,topo,hydroseq
0,wb-824123,nex-824124,05,fl-nex,3565.0


### Converting tables into geopandas dataframes

In [4]:
flowpaths = catalog.load_table("hydrofabric.flowpaths")
to_geopandas(flowpaths)

,id,toid,mainstem,order,hydroseq,lengthkm,areasqkm,tot_drainage_areasqkm,has_divide,divide_id,poi_id,vpuid,geometry
0,wb-20466,nex-20467,2613576.0,1.0,20292,2.443793,4.304250,4.304250,True,cat-20466,19534,01,"MULTILINESTRING ((1873700.565 2684810.523, 187..."
1,wb-20474,nex-20475,2613597.0,1.0,20291,3.982518,4.829400,4.829400,True,cat-20474,None,01,"MULTILINESTRING ((1875378.459 2678583.977, 187..."
2,wb-20475,nex-20468,2613597.0,1.0,20290,1.006645,6.713101,11.542501,True,cat-20475,None,01,"MULTILINESTRING ((1875781.183 2682056.744, 187..."
3,wb-20473,nex-20467,2613587.0,1.0,20289,1.892781,4.486050,4.486050,True,cat-20473,None,01,"MULTILINESTRING ((1875252.558 2684695.906, 187..."
4,wb-20467,nex-20468,2613576.0,2.0,20288,1.616463,2.583900,22.916701,True,cat-20467,1193,01,"MULTILINESTRING ((1875278.618 2683667.5, 18754..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
828283,wb-3054945,nex-3054946,2523667.0,2.0,236,3.515287,9.335699,84.047850,True,cat-3054945,None,17,"MULTILINESTRING ((-2219263.794 2463211.731, -2..."
828284,wb-3054946,nex-3054947,2523667.0,2.0,235,4.365587,13.306500,97.354350,True,cat-3054946,None,17,"MULTILINESTRING ((-2222330.448 2464615.711, -2..."
828285,wb-3054961,nex-3054958,2523700.0,1.0,234,2.032194,5.235750,5.235750,True,cat-3054961,None,17,"MULTILINESTRING ((-2218810.457 2467656.224, -2..."
828286,wb-3054958,nex-3054959,2523693.0,1.0,233,6.466212,10.670850,15.906600,True,cat-3054958,None,17,"MULTILINESTRING ((-2217798.39 2468518.649, -22..."
